<a href="https://colab.research.google.com/github/shubhanginimbalkar/AutoML_Project/blob/main/MLJar_Pro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mljar-supervised
!pip install pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load dataset with dtype specified for problematic columns
file_path = '/content/HomeC.csv'

# Force columns with mixed types to be read as strings
data = pd.read_csv("/content/HomeC.csv", dtype={'column_name': 'str'}, low_memory=False)

# Downcast numerical columns to save memory
def downcast(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64"]
    df[float_cols] = df[float_cols].astype("float32")
    df[int_cols] = df[int_cols].astype("int32")
    return df

data = downcast(data)

# Handle categorical columns with Label Encoding instead of One-Hot Encoding
label_encoder = LabelEncoder()
categorical_columns = data.select_dtypes(include=['object']).columns

for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col].astype(str))  # Ensure consistency in data type

# Verify memory usage reduction
print(data.info())



ParserError: Error tokenizing data. C error: Expected 32 fields in line 3971, saw 50


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load dataset with dtype specified for problematic columns
file_path = '/content/HomeC.csv'

# Force columns with mixed types to be read as strings,
# and handle inconsistent number of columns
try:
    data = pd.read_csv("/content/HomeC.csv", dtype={'column_name': 'str'}, low_memory=False)
except pd.errors.ParserError:
    # If ParserError occurs due to inconsistent number of columns, use error_bad_lines
    data = pd.read_csv("/content/HomeC.csv", dtype={'column_name': 'str'}, low_memory=False, error_bad_lines=False)
    print("Warning: Rows with inconsistent number of columns were skipped.")

# Downcast numerical columns to save memory
def downcast(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64"]
    df[float_cols] = df[float_cols].astype("float32")
    df[int_cols] = df[int_cols].astype("int32")
    return df

data = downcast(data)

# Handle categorical columns with Label Encoding instead of One-Hot Encoding
label_encoder = LabelEncoder()
categorical_columns = data.select_dtypes(include=['object']).columns

for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col].astype(str))  # Ensure consistency in data type

# Verify memory usage reduction
print(data.info())

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [ ]:
# Step 1: Check for missing values
print(data.isnull().sum())

# Step 2: Impute missing values in the feature set and target variable
# Fill missing values with the mean for numerical columns
data.fillna(data.mean(), inplace=True)

# Verify that there are no more missing values
print(data.isnull().sum())

# Redefine X and y after imputation
X = data.drop(columns=[target])
y = data[target]

# Step 3: Split the data into train and test sets again (since we've modified the data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training set size: {X_train.shape}')
print(f'Test set size: {X_test.shape}')


time                   0
use [kW]               0
gen [kW]               0
House overall [kW]     1
Dishwasher [kW]        1
Furnace 1 [kW]         1
Furnace 2 [kW]         1
Home office [kW]       1
Fridge [kW]            1
Wine cellar [kW]       1
Garage door [kW]       1
Kitchen 12 [kW]        1
Kitchen 14 [kW]        1
Kitchen 38 [kW]        1
Barn [kW]              1
Well [kW]              1
Microwave [kW]         1
Living room [kW]       1
Solar [kW]             1
temperature            1
icon                   0
humidity               1
visibility             1
summary                0
apparentTemperature    1
pressure               1
windSpeed              1
cloudCover             0
windBearing            1
precipIntensity        1
dewPoint               1
precipProbability      1
dtype: int64
time                   0
use [kW]               0
gen [kW]               0
House overall [kW]     0
Dishwasher [kW]        0
Furnace 1 [kW]         0
Furnace 2 [kW]         0
Home office 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

df = pd.read_csv(
    "/content/HomeC.csv",
    skipinitialspace=True,
)

# Convert all columns to numeric, replacing non-convertible values with NaN
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            print(f"Could not convert column {col} to numeric")

X_train, X_test, y_train, y_test = train_test_split(
    df[df.columns[:-1]], df['use [kW]'], test_size=0.25
)

automl = AutoML()
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

<ipython-input-5-cc70cccc7033>:5: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.


Linear algorithm was disabled.
AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 0.66941 trained in 0.89 seconds
2_DecisionTree rmse 0.138533 trained in 16.23 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 0.042588 trained in 46.55 seconds
4_Default_NeuralNetwork rmse 0.001412 trained in 10.18 seconds
5_Default_RandomForest rmse 0.069278 trained in 187.79 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 0.001412 trained in 0.42 seconds
AutoML fit time: 273.35 seconds
AutoML best model: 4_Default_NeuralNetwork


In [ ]:
from sklearn.model_selection import train_test_split

# Define the target column (adjust it based on your objective, e.g., 'use [kW]')
target = 'use [kW]'

# Split the data into features (X) and target (y)
X = data.drop(columns=[target])
y = data[target]

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training set size: {X_train.shape}')
print(f'Test set size: {X_test.shape}')

# Initialize AutoML model
automl = AutoML()

# Fit the model on the training data
automl.fit(X_train, y_train)

# Make predictions on the test set
predictions = automl.predict(X_test)

# Ensure predictions do not contain NaNs
predictions = np.nan_to_num(predictions)

# Calculate R² score
r2 = r2_score(y_test, predictions)

# Print R² value
print(f"R² score: {r2}")


Training set size: (403128, 31)
Test set size: (100783, 31)
Linear algorithm was disabled.
AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models


/usr/local/lib/python3.10/dist-packages/supervised/preprocessing/exclude_missing_target.py:25: UserWarning: There are samples with missing target values in the data which will be excluded for further analysis
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 


AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 1.060787 trained in 1.41 seconds
2_DecisionTree rmse 0.188794 trained in 16.85 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 0.037737 trained in 552.3 seconds
4_Default_NeuralNetwork rmse 0.002001 trained in 25.1 seconds
5_Default_RandomForest rmse 0.082374 trained in 171.75 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 0.002001 trained in 0.62 seconds
AutoML fit time: 782.36 seconds
AutoML best model: 4_Default_NeuralNetwork
R² score: 0.9999964285774284


In [ ]:
# Inspect the leaderboard to check for column names
leaderboard = automl.get_leaderboard()

# Print the leaderboard columns
print(leaderboard.columns)

# Print the top 5 rows of the leaderboard to inspect
print(leaderboard.head())


NameError: name 'automl' is not defined

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score

# Get the leaderboard of models from AutoML
leaderboard = automl.get_leaderboard()

# Inspect the leaderboard to check the available models
print(leaderboard)

# Get the top 5 models from the leaderboard
top_models = leaderboard['name'].values[:5]  # Assuming 'name' is the correct column for model names

# Dictionary to store R² values for each model
r2_scores = {}

# Loop over each model and calculate R² score
for model_name in top_models:
    # Get predictions from the specific model (AutoML automatically has them available)
    predictions = automl.predict(X_test)  # AutoML always uses the best model for predictions

    # Calculate R² score
    r2 = r2_score(y_test, predictions)

    # Store the R² value
    r2_scores[model_name] = r2

# Print R² values for all top 5 models
for model_name, r2 in r2_scores.items():
    print(f"R² score for model {model_name}: {r2}")


                      name      model_type metric_type  metric_value  \
0               1_Baseline        Baseline        rmse      1.060787   
1           2_DecisionTree   Decision Tree        rmse      0.188794   
2        3_Default_Xgboost         Xgboost        rmse      0.037737   
3  4_Default_NeuralNetwork  Neural Network        rmse      0.002001   
4   5_Default_RandomForest   Random Forest        rmse      0.082374   
5                 Ensemble        Ensemble        rmse      0.002001   

   train_time  
0        2.35  
1       18.11  
2      553.97  
3       26.18  
4      173.17  
5        0.62  
R² score for model 1_Baseline: 0.9999964285774284
R² score for model 2_DecisionTree: 0.9999964285774284
R² score for model 3_Default_Xgboost: 0.9999964285774284
R² score for model 4_Default_NeuralNetwork: 0.9999964285774284
R² score for model 5_Default_RandomForest: 0.9999964285774284


In [ ]:
pip install matplotlib


In [ ]:
import matplotlib.pyplot as plt

# Data
models = ['Baseline', 'DecisionTree', 'Default_Xgboost', 'Default_NeuralNetwork', 'Default_RandomForest']
scores = [0.9999964285774284] * len(models)  # All models have the same score
times = [2.35, 18.11, 553.97, 26.18, 173.17]

# Create figure and axes
fig, ax1 = plt.subplots()

# Create bar plot for scores
ax1.bar(models, scores, color='b', alpha=0.6, label='Score')
ax1.set_ylabel('Score', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create second y-axis for times
ax2 = ax1.twinx()
ax2.plot(models, times, color='r', marker='o', label='Time (s)')
ax2.set_ylabel('Time (s)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Title and show
plt.title('Model Performance Comparison')
fig.tight_layout()  # Adjust layout
plt.show()


In [ ]:
!python graph.py


python3: can't open file '/content/graph.py': [Errno 2] No such file or directory


In [ ]:
import matplotlib.pyplot as plt

# Data
models = ['Baseline', 'DecisionTree', 'Default_Xgboost', 'Default_NeuralNetwork', 'Default_RandomForest']
scores = [0.9999964285774284] * len(models)  # All models have the same score
times = [2.35, 18.11, 553.97, 26.18, 173.17]

# Create figure and axes
fig, ax1 = plt.subplots()

# Create bar plot for scores
ax1.bar(models, scores, color='b', alpha=0.6, label='Score')
ax1.set_ylabel('Score', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create second y-axis for times
ax2 = ax1.twinx()
ax2.plot(models, times, color='r', marker='o', label='Time (s)')
ax2.set_ylabel('Time (s)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Title and show
plt.title('Model Performance Comparison')
fig.tight_layout()  # Adjust layout
plt.show()


In [ ]:
!python graph.py

python3: can't open file '/content/graph.py': [Errno 2] No such file or directory


In [ ]:
!python plot_models.py


python3: can't open file '/content/plot_models.py': [Errno 2] No such file or directory


In [ ]:
import matplotlib.pyplot as plt

# x-coordinates of left sides of bars
left = [1, 2, 3, 4, 5]

# heights of bars
height = [10, 24, 36, 40, 5]

# labels for bars
tick_label = ['Baseline', 'DecisionTree', 'Default_Xgboost', 'Default_NeuralNetwork', 'Default_RandomForest']

# plotting a bar chart
plt.bar(left, height, tick_label = tick_label,
        width = 0.8, color = ['red', 'green'])

# naming the x-axis
plt.xlabel('x - axis')
# naming the y-axis
plt.ylabel('y - axis')
# plot title
plt.title('My bar chart!')

# function to show the plot
plt.show()
